## The Augmented LLM
In this workbook we'll build a sample workflow of an Augmented LLM, like the one shown in the image below:

<div align="center">
<img src="images/01-Augmented-LLM.png" width="300" heigh="250" alt="Augmented LLM"/>
</div>